In [80]:
from google.colab import files
uploaded=files.upload()

Saving customers (1).csv to customers (1).csv


# Reading csv files using pandas

In [81]:
import pandas as pd
cf=pd.read_csv("customers.csv")
cf.head()


,customer_id,name,email,phone
0,1,Alice Johnson,alice.johnson@example.com,555-0101
1,2,Bob Smith,bob.smith@example.com,555-0202
2,3,Carol Lee,carol.lee@example.com,555-0303
3,4,David Patel,david.patel@example.com,555-0404
4,5,Eva Wang,eva.wang@example.com,555-0505


In [82]:
of=pd.read_csv("orders.csv")
of.head()


,order_id,customer_id,order_date,total_amount,delivery_date
0,1001,1,2025-07-01,120.50,2025-07-03
1,1002,2,2025-07-03,75.00,2025-07-05
2,1003,3,2025-07-05,200.00,2025-07-07
3,1004,1,2025-07-07,50.00,2025-07-09
4,1005,4,2025-07-10,315.75,2025-07-12


In [83]:
df=pd.read_csv("deliveries.csv")
df.head()

,status_id,order_id,status,status_time,remarks
0,1,1001,pending,2025-07-02 08:00:00,left warehouse
1,2,1001,shipped,2025-07-03 09:30:00,departed warehouse
2,3,1001,delivered,2025-07-05 14:15:00,signed by customer
3,4,1002,pending,2025-07-04 10:00:00,order received
4,5,1002,shipped,2025-07-05 11:00:00,picked up by carrier


In [89]:
cf=cf.dropna(subset=["customer_id","email","phone"])
cf.isnull().sum()
with open("customers_cleaned.csv","w") as f:
  cf.to_csv(f)


In [85]:
of['total_amount']=of['total_amount'].fillna(0)
of=of.dropna()
of.isnull().sum()

,0
order_id,0
customer_id,0
order_date,0
total_amount,0
delivery_date,0


In [90]:
df['remarks']=df['remarks'].fillna("Nil")
df=df.dropna(subset=["status_id","order_id"])
df['status'] = df['status'].fillna('pending')

df.isnull().sum()
with open("deliveries_cleaned.csv","w") as f:
  df.to_csv(f)

# converting time stamps

In [91]:
of['order_date']=pd.to_datetime(of['order_date'],errors="coerce")
of['delivery_date']=pd.to_datetime(of['delivery_date'],errors="coerce")
df['status_time']=pd.to_datetime(df['status_time'],errors="coerce")

# if conversion failed

df = df.dropna(subset=["status_time"])
with open("orders_cleaned.csv","w") as f:
  of.to_csv(f)


# using numpy to calculate delivery delays


In [96]:
import numpy as np
merged_df=pd.merge(df,of,on="order_id",how="left")
delayed=merged_df[merged_df['status']=="delayed"]
merged_df["delayed_Days"]=(merged_df["status_time"]-merged_df["delivery_date"]).dt.days
merged_df["delayed"]=np.where(merged_df["delayed_Days"]>0,1,0)

delayed_orders=merged_df[merged_df['delayed']==1]
extracted_delays=delayed_orders[["status_id","order_id","customer_id","delivery_date","delayed_Days","delayed"]]
print(extracted_delays)
with open("extracted_delays.csv","w") as f:
  extracted_delays.to_csv(f)

    status_id  order_id  customer_id delivery_date  delayed_Days  delayed
2           3      1001            1    2025-07-03             2        1
5           6      1002            2    2025-07-05             3        1
8           9      1003            3    2025-07-07             2        1
11         12      1004            1    2025-07-09             2        1
14         15      1005            4    2025-07-12             6        1
17         18      1006            2    2025-07-13             1        1
20         21      1007            5    2025-07-14             6        1
23         24      1008            3    2025-07-16             1        1
26         27      1009            4    2025-07-17             2        1
29         30      1010            5    2025-07-18             1        1


#  Show top delayed customers

In [95]:
top_delayed_customers=extracted_delays.groupby('customer_id')['delayed'].sum().sort_values(ascending=False)
top_delayed_customers.head(5)

,delayed
customer_id,
1,2
2,2
3,2
4,2
5,2


# most common delivery issues

In [94]:
issues=merged_df['status'].value_counts()
print("the most common delivery issues:\n")
print(issues)

the most common delivery issues:

status
pending      10
shipped      10
delivered     7
delayed       3
Name: count, dtype: int64
